In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 KB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 
import os
import numpy as np 

import snscrape.modules.twitter as sntwitter 

import datetime 

from tqdm.notebook import tqdm_notebook 

import seaborn as sns 

import matplotlib.pyplot as plt 

sns.set_theme(style="whitegrid") 

In [ ]:
lst_terms = ["#turkeyearthquake2023", "#TurkeySyriaEarthquake2023", "turkey earthquakes", "#TurkeySyriaEarthquakes","#syriaearthquake","#syriaearthquake2023","syria earthquakes","turkey relief","syria earthquake relief"]
since = "2023-02-05"
until = ""
count = -1
retweet = 'n'
replies = 'n'

chunk = 2048

In [ ]:
print(lst_terms)
print(since)
print(until)
print(count)
print(retweet)
print(replies)

['#turkeyearthquake2023', '#TurkeySyriaEarthquake2023', 'turkey earthquakes', '#TurkeySyriaEarthquakes', '#syriaearthquake', '#syriaearthquake2023', 'syria earthquakes', 'turkey relief', 'syria earthquake relief']
2023-02-05

-1
n
n


In [ ]:
def createConf(text):
    FOLDER_NAME = "bucket_" + text
    if not os.path.exists(FOLDER_NAME):
        os.makedirs(FOLDER_NAME)
    with open(os.path.join(FOLDER_NAME,"conf.txt"), "w") as file1:
        toFile = text + "\n" + str(since) + "\n" + str(until) + "\n" + str(count) + "\n" + str(retweet) + "\n" + str(replies)
        file1.write(toFile)

In [ ]:
def search(text,since,until,retweet,replies):
    global filename
    q = text
    # if username!='':
    #     q += f" from:{username}"    
    if until=='':
        until = datetime.datetime.strftime(datetime.date.today(), '%Y-%m-%d')
    q += f" until:{until}"
    if since=='':
        since = datetime.datetime.strftime(datetime.datetime.strptime(until, '%Y-%m-%d') - 
                                           datetime.timedelta(days=7), '%Y-%m-%d')
    q += f" since:{since}"
    if retweet == 'y':
        q += f" exclude:retweets"
    if replies == 'y':
        q += f" exclude:replies"
    # if username!='' and text!='':
    #     filename = f"{since}_{until}_{username}_{text}.csv"
    # elif username!="":
    #     filename = f"{since}_{until}_{username}.csv"
    # else:
    #     filename = f"{since}_{until}_{text}.csv"
    # print(filename)
    return q

In [ ]:
def saveData(tweets_list1, base=""):
    tweets_df1 = pd.DataFrame(tweets_list1, columns=['DateTime', 'TweetId', 'Text', 'Username','Language',
                                    'Hashtags','ReplyCount','RetweetCount','LikeCount','QuoteCount','Media'])
    import uuid
    tweets_df1.sort_values(by='DateTime',ascending=False)
    tweets_df1.to_csv(os.path.join(base, "tqdm-tweets-"+str(uuid.uuid4())+".csv"))

In [ ]:
for text in lst_terms:
    createConf(text)
    tweets_list1 = []
    FOLDER_NAME = "bucket_" + text
    try:
        q = search(text,since,until,retweet,replies)
        print(q)
        # Creating list to append tweet data

        # Using TwitterSearchScraper to scrape data and append tweets to list
        if count == -1:
            for i,tweet in enumerate(tqdm_notebook(sntwitter.TwitterSearchScraper(q).get_items())):
                tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username,tweet.lang,
                tweet.hashtags,tweet.replyCount,tweet.retweetCount,tweet.likeCount,tweet.quoteCount,tweet.media])

                if(chunk != -1 and len(tweets_list1) > chunk):
                    saveData(tweets_list1, base=FOLDER_NAME)
                    tweets_list1 = []
        else:
            with tqdm_notebook(total=count) as pbar:
                for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()): #declare a username 
                    if i>=count: #number of tweets you want to scrape
                        break
                    tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username,tweet.lang,
                tweet.hashtags,tweet.replyCount,tweet.retweetCount,tweet.likeCount,tweet.quoteCount,tweet.media])
                    pbar.update(1)
        # Creating a dataframe from the tweets list above 
    except KeyboardInterrupt:
         print("Keyboard interrupt exception caught")

    saveData(tweets_list1)

#turkeyearthquake2023 until:2023-03-20 since:2023-02-05


0it [00:00, ?it/s]

#TurkeySyriaEarthquake2023 until:2023-03-20 since:2023-02-05


0it [00:00, ?it/s]

turkey earthquakes until:2023-03-20 since:2023-02-05


0it [00:00, ?it/s]